In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from skimage import draw
from skimage import io
import numpy as np
import urllib.request
import json
import logging
import os
import sys
# !pip3 install opencv-python
import cv2
# !pip3 install pandas
import pandas as pd



def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images
folder = '/content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/train'
images = load_images_from_folder(folder)

In [ ]:
images[0].shape

(590, 1042, 3)

In [ ]:
def load_image_names_from_folder(folder):
    image_names = []
    k=  os.listdir(folder)
    return k
folder = '/content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/train'
img_names= load_image_names_from_folder(folder)

In [ ]:
img_names

['2_Ave_12.jpg',
 '2_Ave_95.jpg',
 '2_Ave_96.jpg',
 '2_Ave_97.jpg',
 '2_Ave_124.jpg',
 '2_Ave_17.jpg',
 '2_Ave_18.jpg',
 '2_Ave_117.jpg',
 '2_Ave_19.jpg',
 '2_Ave_118.jpg',
 '2_Ave_123.jpg',
 '2_Ave_20.jpg',
 '2_Ave_14.jpg',
 '2_Ave_2.jpg',
 '2_Ave_99.jpg',
 '2_Ave_119.jpg',
 '2_Ave_116.jpg',
 '2_Ave_15.jpg',
 '2_Ave_98.jpg',
 '2_Ave_125.jpg',
 'Queens_Blvd_95.jpg',
 'South_St_522.jpg',
 'Queens_Blvd_92.jpg',
 'South_St_521.jpg',
 'Queens_Blvd_9.jpg',
 'South_St_520.jpg',
 'South_St_511.jpg',
 '2_Ave_21.jpg',
 '2_Ave_24.jpg',
 'Queens_Blvd_97.jpg',
 'Queens_Blvd_89.jpg',
 '2_Ave_22.jpg',
 'Queens_Blvd_91.jpg',
 'Queens_Blvd_93.jpg',
 'Queens_Blvd_90.jpg',
 'South_St_514.jpg',
 '2_Ave_25.jpg',
 'South_St_513.jpg',
 'South_St_517.jpg',
 '2_Ave_23.jpg',
 'Bath_Ave_41.jpg',
 'Queens_Blvd_96.jpg',
 'South_St_531.jpg',
 'South_St_532.jpg',
 'Queens_Blvd_21.jpg',
 'Queens_Blvd_109.jpg',
 'South_St_533.jpg',
 'Queens_Blvd_34.jpg',
 'Bath_Ave_60.jpg',
 'Queens_Blvd_157.jpg',
 'Queens_Blvd_50.jp

In [ ]:
logging.getLogger().setLevel(logging.INFO)

In [ ]:
df = pd.read_json("/content/drive/My Drive/5_corridors_train_val_data/via_region_data_train.json", orient='rows').T
df.head(5)
df.drop(["fileref","file_attributes","size"], axis = 1, inplace = True) 
df.head(93)

,filename,regions
South_St_525.jpg,South_St_525.jpg,"{'6': {'shape_attributes': {'name': 'polygon',..."
7_Ave_S_422.jpg,7_Ave_S_422.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."
7_Ave_S_172.jpg,7_Ave_S_172.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."
Yellowstone_Blvd_144.jpg,Yellowstone_Blvd_144.jpg,"{'2': {'shape_attributes': {'name': 'polygon',..."
7_Ave_S_106.jpg,7_Ave_S_106.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."
...,...,...
2nd_Ave_361.jpg,2nd_Ave_361.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."
7_Ave_S_28.jpg,7_Ave_S_28.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."
7_Ave_S_148.jpg,7_Ave_S_148.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."
2nd_Ave_1.jpg,2nd_Ave_1.jpg,"{'0': {'shape_attributes': {'name': 'polygon',..."


In [ ]:
df.shape

(847, 2)

In [ ]:
file_names =list(df['filename'])
regions = list(df['regions'])

In [ ]:
regions[92]

{'0': {'region_attributes': {'class': 1},
  'shape_attributes': {'all_points_x': [242, 828, 961, 963, 92, 242],
   'all_points_y': [312, 307, 343, 356, 401, 312],
   'name': 'polygon'}}}

In [ ]:
def poly2mask(blobs_for_current_image, c, path_to_masks_folder, h, w,objects):
    mask = np.zeros((h, w))
    for j in range(len(blobs_for_current_image)):
        fill_row_coords, fill_col_coords = draw.polygon(np.array(blobs_for_current_image[j][1]), np.array(blobs_for_current_image[j][0]))
        mask[fill_row_coords, fill_col_coords] = 1
    io.imsave(path_to_masks_folder + "/" + str(objects), mask)

In [ ]:
def convert_jason_to_masks(path_to_annotation_json, path_to_original_images_folder, path_to_masks_folder):
    # make sure everything is setup.
    if (not os.path.isdir(path_to_original_images_folder)):
        logging.exception(
            "Please specify a valid directory path to download images, " + path_to_original_images_folder + " doesn't exist")
        return
    print('check 1...........:')
    if (not os.path.isdir(path_to_masks_folder)):
        logging.exception(
            "Please specify a valid directory path to write mask files, " + path_to_masks_folder + " doesn't exist")
        return
    print('check 2...........:')
    if (not os.path.exists(path_to_annotation_json)):
        logging.exception(
            "Please specify a valid path to dataturks JSON output file, " + path_to_dataturks_annotation_json + " doesn't exist")
        return
    print('check 3...........:')

    c = 0
    index= 0
    for objects in file_names:
#         blobs = []
#         classes = {}
        height=768
        width = 1366
        
        
        annotations = regions[index]
        index+=1

       
        
        
        keys =[]
        for key in annotations:
            keys.append(key)
        blobs_for_current_image = []
        for i in keys:
            x = annotations[str(i)]['shape_attributes']['all_points_x']
            y = annotations[str(i)]['shape_attributes']['all_points_y']
            label = annotations[str(i)]['region_attributes']['class']
#             classes.append(label)
            # print(blobs_for_current_image)
            blobs_for_current_image.append([x,y,height,width,label])

#         blobs.append(blobs_for_current_image)
#         print(blobs[0])    
#         print('xxxxxxxxxxxxxxxxxxxxxxx',x)   
#         print(objects)
        print('image name::  ',objects)
        poly2mask(blobs_for_current_image, c, path_to_masks_folder, height,width,objects)
        c += 1

convert_jason_to_masks('/content/drive/My Drive/5_corridors_train_val_data/via_region_data_train.json', '/content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/train', '/content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/mask')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: /content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/mask/South_St_525.jpg is a low contrast image
  


check 1...........:
check 2...........:
check 3...........:
image name::   South_St_525.jpg
image name::   7_Ave_S_422.jpg
image name::   7_Ave_S_172.jpg


image name::   Yellowstone_Blvd_144.jpg
image name::   7_Ave_S_106.jpg
image name::   7_Ave_S_300.jpg


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: /content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/mask/2nd_Ave_23.jpg is a low contrast image
  


image name::   7_Ave_S_388.jpg
image name::   7_Ave_S_305.jpg
image name::   2nd_Ave_23.jpg
image name::   South_St_890.jpg


...

image name::   South_St_564.jpg
image name::   7_Ave_S_139.jpg
image name::   7_Ave_S_410.jpg
image name::   7_Ave_S_194.jpg


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: /content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/mask/Yellowstone_Blvd_220.jpg is a low contrast image
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: /content/drive/My Drive/5_corridors_train_val_data/train-20200530T092227Z-001/mask/7_Ave_S_70.jpg is a low contrast image
  


image name::   Queens_Blvd_215.jpg
image name::   Yellowstone_Blvd_220.jpg
image name::   7_Ave_S_70.jpg
image name::   7_Ave_S_349.jpg
image name::   E_Tremont_78.jpg


image name::   7_Ave_S_20.jpg
image name::   2nd_Ave_308.jpg
image name::   2nd_Ave_44.jpg
image name::   South_St_144.jpg


image name::   7_Ave_S_159.jpg
image name::   7_Ave_S_165.jpg
image name::   7_Ave_S_179.jpg


some other stuff

In [ ]:
import cv2
 
# read image
img = cv2.imread('/content/drive/My Drive/5_corridors_train_val_data/train/Riverside_Drive_178.jpg', cv2.IMREAD_UNCHANGED)
 
# get dimensions of image
dimensions = img.shape
 
# height, width, number of channels in image
height = img.shape[0]
width = img.shape[1]
channels = img.shape[2]
 
print('Image Dimension    : ',dimensions)
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Channels : ',channels)

Image Dimension    :  (590, 1042, 3)
Image Height       :  590
Image Width        :  1042
Number of Channels :  3


In [ ]:
import cv2
 
# read image
img = cv2.imread('/content/drive/My Drive/5_corridors_train_val_data/train/2nd_Ave_51.jpg', cv2.IMREAD_UNCHANGED)
 
# get dimensions of image
dimensions = img.shape
 
# height, width, number of channels in image
height = img.shape[0]
width = img.shape[1]
channels = img.shape[2]
 
print('Image Dimension    : ',dimensions)
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Channels : ',channels)